In [8]:
import pandas as pd
import numpy as np
from time import sleep
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os


In [9]:
#Librerias de Selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [10]:
#Hacemos que no muestre los warnings
import warnings
warnings.filterwarnings("ignore")


In [11]:
lista_supermercados= ["mercadona", "carrefour","eroski", "dia", "hipercor", "alcampo"]
lista_productos=["aceite-de-girasol", "aceite-de-oliva", "leche"]

In [12]:
def esperar_y_hacer_click(drivers, ruta_xpath):
    """
    Espera hasta que un elemento esté presente en la página y realiza un clic en él.

    Args:
        drivers (WebDriver): El objeto del navegador (WebDriver) que está siendo utilizado para controlar la página.
        ruta_xpath (str): El XPath del elemento que se está buscando en la página.

    Returns:
        WebElement: El botón o elemento sobre el cual se hizo clic. Retorna `None` si el elemento no se encuentra después de 5 intentos.
    """
    
    boton = None
    intentos = 0
    while True and intentos <= 5:
        try:
            boton = WebDriverWait(drivers, 10).until(EC.presence_of_element_located(("xpath", ruta_xpath)))
            sleep(2)
            boton.click()
            break
        except Exception as e:
            intentos += 1
            print(f"Intento {intentos}: No se ha podido encontrar el botón. Error: {e}")
            continue
    
    return boton

In [13]:
driver = webdriver.Chrome()

url = f"https://super.facua.org/{lista_supermercados[0]}/{lista_productos[0]}"
driver.get(url)
driver.maximize_window()
sleep(2)
# Acceptamso cookies
esperar_y_hacer_click(driver, '//*[@id="rcc-confirm-button"]')
driver.execute_script("window.scrollTo(0, 300)")
# Entramos en los aceites de girasol
esperar_y_hacer_click(driver, '/html/body/section[1]/div/div[3]/div[1]/div/div[2]/div/a')
# Extraemos informacion html
sleep(1)
contenido_html = driver.page_source
#driver.close()
sopa = BeautifulSoup(contenido_html, 'html.parser')
tabla = sopa.find("table")
tabla
    

<table class="table table-striped table-responsive text-center" style="width:100%"><thead><tr><th scope="col">Día</th><th scope="col">Precio (€)</th><th scope="col">Variación</th></tr></thead><tbody><tr><td>12/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>13/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>14/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>15/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>16/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>17/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>18/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>19/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>20/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>21/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>22/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>23/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>24/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>25/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>26/07/2024</td><td>1,45</td><td>=</td></tr><tr><td>27/07/2024</td><td>1,45</td>

## Lo estas enfocando de manera muy poco eficaz. Saca las URL de cada producto. Y escrapea la tabla correspondiente. Una vez que tienes la tabla escrapea (usando la url) el nombre de cada producto haz lo mismo con las categorias y sub categorias (girasol, oliva, oliva virgen, etc) y con eso te construyes la  base de datos

In [14]:
lista_encabezados = tabla.findAll("th")

In [15]:
lista_encabezados

[<th scope="col">Día</th>,
 <th scope="col">Precio (€)</th>,
 <th scope="col">Variación</th>]

In [16]:
for i, encabezado in enumerate(lista_encabezados):
    print(f"El encabezado {i+1} es: {encabezado.getText()}")

El encabezado 1 es: Día
El encabezado 2 es: Precio (€)
El encabezado 3 es: Variación


In [17]:
lista_filas = tabla.findAll("tr")

In [18]:
lista_filas

[<tr><th scope="col">Día</th><th scope="col">Precio (€)</th><th scope="col">Variación</th></tr>,
 <tr><td>12/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>13/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>14/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>15/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>16/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>17/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>18/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>19/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>20/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>21/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>22/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>23/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>24/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>25/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>26/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>27/07/2024</td><td>1,45</td><td>=</td></tr>,
 <tr><td>28/07/2024</td><td>1,45</td><t

In [19]:
lista_filas = lista_filas[1:]
datos_tabla = []
for fila in lista_filas:
    # Usamos BeautifulSoup para convertir cada fila en una estructura de HTML
    fila_bs = BeautifulSoup(str(fila), 'html.parser')
    celdas = fila_bs.find_all('td')
    # Extraemos el texto de cada celda y lo añadimos a la lista
    datos_tabla.append([celdas[0].text, celdas[1].text.replace(',', '.'), celdas[2].text])
    
datos_tabla



[['12/07/2024', '1.45', '='],
 ['13/07/2024', '1.45', '='],
 ['14/07/2024', '1.45', '='],
 ['15/07/2024', '1.45', '='],
 ['16/07/2024', '1.45', '='],
 ['17/07/2024', '1.45', '='],
 ['18/07/2024', '1.45', '='],
 ['19/07/2024', '1.45', '='],
 ['20/07/2024', '1.45', '='],
 ['21/07/2024', '1.45', '='],
 ['22/07/2024', '1.45', '='],
 ['23/07/2024', '1.45', '='],
 ['24/07/2024', '1.45', '='],
 ['25/07/2024', '1.45', '='],
 ['26/07/2024', '1.45', '='],
 ['27/07/2024', '1.45', '='],
 ['28/07/2024', '1.45', '='],
 ['29/07/2024', '1.45', '='],
 ['30/07/2024', '1.45', '='],
 ['31/07/2024', '1.45', '='],
 ['01/08/2024', '1.45', '='],
 ['02/08/2024', '1.45', '='],
 ['03/08/2024', '1.45', '='],
 ['04/08/2024', '1.45', '='],
 ['05/08/2024', '1.45', '='],
 ['06/08/2024', '1.45', '='],
 ['07/08/2024', '1.45', '='],
 ['08/08/2024', '1.45', '='],
 ['09/08/2024', '1.45', '='],
 ['10/08/2024', '1.45', '='],
 ['11/08/2024', '1.45', '='],
 ['12/08/2024', '1.45', '='],
 ['13/08/2024', '1.45', '='],
 ['14/08/2

In [20]:
lista_encabezados

[<th scope="col">Día</th>,
 <th scope="col">Precio (€)</th>,
 <th scope="col">Variación</th>]

In [21]:
# Extraemos el texto de cada elemento <th>
encabezados = [BeautifulSoup(str(th), 'html.parser').text for th in lista_encabezados]

# Mostramos la lista de encabezados
encabezados


['Día', 'Precio (€)', 'Variación']

In [22]:
df = pd.DataFrame(datos_tabla, columns= encabezados)
df


,Día,Precio (€),Variación
0,12/07/2024,1.45,=
1,13/07/2024,1.45,=
2,14/07/2024,1.45,=
3,15/07/2024,1.45,=
4,16/07/2024,1.45,=
...,...,...,...
101,21/10/2024,1.48,=
102,22/10/2024,1.48,=
103,23/10/2024,1.48,=
104,24/10/2024,1.48,=
